## baseline implementation

#### dataset：
https://ciir.cs.umass.edu/downloads/nfL6/

todo: use another dataset

#### baselines: 
1) random choose     
2) SVM as classifier      
3) learning to rank(LexRank) reference: http://tangra.si.umich.edu/%7Eradev/lexrank/lexrank.pdf             
4) Luhn - heurestic method   reference: http://ieeexplore.ieee.org/document/5392672/?arnumber=5392672           

todo: 
CNN

#### evaluation
ROUGE-1    
ROUGE-2    
ROUGE-L     


## dataset_1

In [113]:
import json

In [114]:
mydata = json.load(open('/Users/zhangyiman/Desktop/nfL6.json','r'))

In [115]:
#mydata[0:2]

In [116]:
questions = []
bestAnswers = []
NotBestAnswers = []
main_categorys = []
q_id = []

In [117]:
for q_a in mydata:
    questions.append(q_a['question'])
    bestAnswers.append(q_a['answer'])
    NotBestAnswers.append(q_a['nbestanswers'])
    main_categorys.append(q_a['main_category'])
    q_id.append(q_a['id'])
    

In [118]:
questions[0]

'Why did the U.S Invade Iraq ?'

In [119]:
bestAnswers[0]

"A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 9/11, they were able to use the terrorist attacks to justify war with Iraq on this basis and exaggerated threats of the development of weapons of mass destruction. The military strength of the U.S. and the brutality of Saddam's regime led them to imagine that the military and political victory would be relatively easy."

In [120]:
NotBestAnswers[0]

["A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 9/11, they were able to use the terrorist attacks to justify war with Iraq on this basis and exaggerated threats of the development of weapons of mass destruction. The military strength of the U.S. and the brutality of Saddam's regime led them to imagine that the military and political victory would be relatively easy.",
 'Because there is a lot of oil in Iraq.',
 'It is tempting to say that the US invaded Iraq because it has lots of oil, but the US is not a country in a deep economic problem that capturing other country’s oil is an actual need for survival. It is more likely that the Iraq invading Kuwait scenario would fall under that assumption.. I think that the US government has come to a concl

In [121]:
main_categorys[0]

'News & Events'

In [122]:
# sentences candidate

query_id = []
query = []
category = []
sentence_id = []
sentence = []
whether_groundTruth = []
for j in range (0,len(NotBestAnswers)):
    # for every question, 
    sentence_candidate = []
    Ground_Truth_sentence = []
    
    # there are len_sen answers for this question
    len_sen = len(NotBestAnswers[j])
    for a in range(0,len_sen):
        str = NotBestAnswers[j][a]
        str_split = str.split('. ')
        # reduce if sentence is null
        for i in range(0,len(str_split)):
            if len(str_split[i])>=2:
                sentence_candidate.append(str_split[i])
                
    #Ground Truth sentences
    str = bestAnswers[j]
    str_split = str.split('. ')
    for i in range(0,len(str_split)):
        Ground_Truth_sentence.append(str_split[i])
        
    #Whether ground truth
    for i in range(0,len(sentence_candidate)):
        if sentence_candidate[i] not in Ground_Truth_sentence:
            Ground_Truth = 0
        else:
            Ground_Truth = 1
        query_id.append(j)
        query.append(questions[j])
        category.append(main_categorys[j])
        sentence_id.append(i)
        sentence.append(sentence_candidate[i])
        whether_groundTruth.append(Ground_Truth)
    

## A dataframe for sentence candidate pool: 
transfer the data format into: 
A dataFrame:      

query, main_category, sentence_id, sentence, whether_groudTruth

In [12]:
import csv

with open('sentence_QA.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['query_id','query','main_category','sentence_id','sentence','whether_groundTruth'])
    for i in range(0,len(query_id)):
        writer.writerow([query_id[i],query[i],category[i],sentence_id[i],sentence[i],whether_groundTruth[i]])
    

In [66]:
import pandas as pd
import numpy as np

In [67]:
sentence_data = pd.read_csv('sentence_QA.csv')

In [142]:
sentence_data[0:10]

,query_id,query,main_category,sentence_id,sentence,whether_groundTruth
0,0,Why did the U.S Invade Iraq ?,News & Events,0,A small group of politicians believed strongly...,1
1,0,Why did the U.S Invade Iraq ?,News & Events,1,Shortly after taking power with George Bush in...,1
2,0,Why did the U.S Invade Iraq ?,News & Events,2,The military strength of the U.S,1
3,0,Why did the U.S Invade Iraq ?,News & Events,3,and the brutality of Saddam's regime led them ...,1
4,0,Why did the U.S Invade Iraq ?,News & Events,4,Because there is a lot of oil in Iraq.,0
5,0,Why did the U.S Invade Iraq ?,News & Events,5,It is tempting to say that the US invaded Iraq...,0
6,0,Why did the U.S Invade Iraq ?,News & Events,6,It is more likely that the Iraq invading Kuwai...,0
7,0,Why did the U.S Invade Iraq ?,News & Events,7,I think that the US government has come to a c...,0
8,0,Why did the U.S Invade Iraq ?,News & Events,8,It would be presumptuous to try and determent ...,0
9,0,Why did the U.S Invade Iraq ?,News & Events,9,I think that the world wide spread of the medi...,0


In [131]:
sentence_data.tail(3)

,query_id,query,main_category,sentence_id,sentence,whether_groundTruth
1438096,87361,How can I finance investment property?,Business & Finance,43,Ask your loan officer about this,0
1438097,87361,How can I finance investment property?,Business & Finance,44,They may also be able to offer you other alter...,0
1438098,87361,How can I finance investment property?,Business & Finance,45,Good luck! :),0


# Random select sentences

In [17]:
## random select, random select id of sentences as a subset sentences contained in answer, the length of answer
## is set to the average of groud truth answer. 
## Then use ROUGE to evaluate the performance of random choose sentences. 

### average ground truth answer length

In [18]:
len_a = 0
num = 0
for i in range(0,len(bestAnswers)):
    str = bestAnswers[i]
    str_split = str.split('. ')
    len_a = len_a + len(str_split)
    num = num + 1
avg_len = len_a/i   

In [19]:
avg_len

3.0053914218014905

In [20]:
import random
from collections import defaultdict

In [27]:
sentence_selected = defaultdict(list)
for i in range(0,87362):
    if len(sentence_data[sentence_data['query_id']== i]) > 3: 
        sen_num = random.sample(range(len(sentence_data[sentence_data['query_id']== i])),3)
        for j in range(0,3):
            sub = sentence_data[sentence_data['query_id']==i][sentence_data['sentence_id'] ==sen_num[j]]
            sentence_sel = sub['sentence'].values[0]
            sentence_selected[i].append(sentence_sel)
    else :
        if len(sentence_data[sentence_data['query_id']== i]) >= 1 :
            sen_num = random.sample(range(len(sentence_data[sentence_data['query_id']== i])),1)
            for j in range(0,1):
                sub = sentence_data[sentence_data['query_id']==i][sentence_data['sentence_id'] ==sen_num[j]]
                sentence_sel = sub['sentence'].values[0]
                sentence_selected[i].append(sentence_sel)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [28]:
#from rouge import Rouge
#hypothesis = str
#reference = bestAnswers[2]
#rouge = Rouge()
#scores = rouge.get_scores(reference, hypothesis)
#scores
#scores[0]
#scores[0]['rouge-1']['f']

### ROUGE evaluation

In [190]:
from rouge import Rouge

In [30]:
rouge_1_f_sum = 0
rouge_1_p_sum = 0
rouge_1_r_sum = 0
rouge_2_f_sum = 0
rouge_2_p_sum = 0
rouge_2_r_sum = 0
rouge_l_f_sum = 0
rouge_l_p_sum = 0
rouge_l_r_sum = 0
num = 0
for i in range(0,len(sentence_selected)):
    str_sen = ''
    for j in range(0,len(sentence_selected[i])):
        str_sen = str_sen + sentence_selected[i][j]
    # str_sen
    hypothesis = str_sen
    reference = bestAnswers[i]
    rouge = Rouge()
    scores = rouge.get_scores(reference, hypothesis)
    rouge_1_f = scores[0]['rouge-1']['f']
    rouge_1_p = scores[0]['rouge-1']['p']
    rouge_1_r = scores[0]['rouge-1']['r']
    rouge_2_f = scores[0]['rouge-2']['f']
    rouge_2_p = scores[0]['rouge-2']['p']
    rouge_2_r = scores[0]['rouge-2']['r']
    rouge_l_f = scores[0]['rouge-l']['f']
    rouge_l_p = scores[0]['rouge-l']['p']
    rouge_l_r = scores[0]['rouge-l']['r']
    
    rouge_1_f_sum = rouge_1_f_sum + rouge_1_f
    rouge_1_p_sum = rouge_1_p_sum + rouge_1_p
    rouge_1_r_sum = rouge_1_r_sum + rouge_1_r
    rouge_2_f_sum = rouge_2_f_sum + rouge_2_f
    rouge_2_p_sum = rouge_2_p_sum + rouge_2_p
    rouge_2_r_sum = rouge_2_r_sum + rouge_2_r
    rouge_l_f_sum = rouge_l_f_sum + rouge_l_f
    rouge_l_p_sum = rouge_l_p_sum + rouge_l_p
    rouge_l_r_sum = rouge_l_r_sum + rouge_l_r
    
    num = num + 1

rouge_1_F = rouge_1_f_sum/num
rouge_1_P = rouge_1_p_sum/num
rouge_1_R = rouge_1_r_sum/num
rouge_2_F = rouge_2_f_sum/num
rouge_2_P = rouge_2_p_sum/num
rouge_2_R = rouge_2_r_sum/num
rouge_l_F = rouge_l_f_sum/num
rouge_l_P = rouge_l_p_sum/num
rouge_l_R = rouge_l_r_sum/num

In [31]:
print("rouge_1: f: ")
print(rouge_1_F)
print("rouge_1: p: ")
print(rouge_1_P)
print("rouge_1: r: ")
print(rouge_1_R)

print("rouge_2: f: ")
print(rouge_2_F)
print("rouge_2: p: ")
print(rouge_2_P)
print("rouge_2: r: ")
print(rouge_2_R)
      
print("rouge_L: f: ")
print(rouge_l_F)
print("rouge_L: p: ")
print(rouge_l_P)
print("rouge_L: r: ")
print(rouge_l_R)

rouge_1: f: 
0.3727697344547275
rouge_1: p: 
0.3785854143176447
rouge_1: r: 
0.4102658319113501
rouge_2: f: 
0.27221121931996173
rouge_2: p: 
0.2749610022289372
rouge_2: r: 
0.304958320306709
rouge_L: f: 
0.28202534345593805
rouge_L: p: 
0.31278125318227223
rouge_L: r: 
0.34635741256111297


# SVM as a classifier
### to classify the important sentences and unimportance sentences. 

In [32]:
# load word2vec
# Sentence vector = word+word+...+word

In [7]:
import numpy as np
import csv
#import datetime
from sklearn.svm import SVC
#import os
import pandas as pd
from sklearn import metrics
#cwd = os.getcwd()

### load word2Vec 

In [30]:
f = open('/Users/zhangyiman/Desktop/word2vec_pre/glove.6B.50d.txt')
embeddings_index = {}
count = 0

for line in f:
        values_int = line.split('\t')
        #print(values_int[0])
        #values = values_int[0].split(' ',1)
        values = values_int[0].split(' ')
        id = values[0]
        #print(id)
        #coefs = np.asarray(values[1:])
        coefs = values[1:]
        #print(coefs)
        embeddings_index[id] = coefs
f.close()

### sentence representation

In [13]:
# sentence representation
# simply use the average of word vectors as a sentence vector

# for some word did not exist in the Word2Vec pre trained dataset, i just ignore them.
import re

In [97]:
## add spammer in this part 
def sentence_vector(sentence_str):
    str = sentence_str
    #str = str.replace(',',' ,')
    str = str.replace("'s",'')
    str = str.replace("determent",'deter')
    str = str.replace("200B",'200 b')
    token = re.compile("[\w]+(?=n't)|n't|\'m|\'ll|[\w]+|[.?!;,\-\(\)—\:']")
    tokens = token.findall(str)
    
    #str_convert = ''.join(tokens[0])
    a = [0]*50
    for i in range(0,len(tokens)):
            for j in range(0,50):
                str_convert = ''.join(tokens[i])
                str_lower = str_convert.lower()
                try: 
                    m = float(embeddings_index[str_lower][j])
                except:
                    m = 0
                a[j] = a[j] + m
    for i in range(0,50):
        a[i] = a[i]/len(tokens)
    return(a)

In [80]:
for i in range(0,len(query)):
    str = query[i]
    query[i] = sentence_vector(str)
    str = sentence[i]
    sentence[i] = sentence_vector(str)

In [101]:
print(query[1])
print(query[1][0])

[2.5268430000000004, 1.4918339999999999, 3.135254, 1.0666099999999998, 3.604157, -2.1879049, -0.3421600000000001, 0.22391999999999995, -1.7088212299999996, -1.75622, -0.8935339999999999, 1.8553359999999999, -3.5713099999999995, -1.6307234999999995, 3.5461504999999995, 4.276671, 0.680471, -0.8523640000000001, -1.3991343, -0.42590000000000006, -1.443699, 0.5682040000000002, 3.280732, -0.196004, 1.678147, -15.376930000000003, -5.447674, -0.3224519999999999, 4.207715, -2.92, 24.51156, 1.3777384000000001, -6.00834, -0.6052999999999998, -0.7777028699999999, -2.3512020999999996, 1.620501, -1.8215824999999999, -1.3276729999999999, -1.3349399999999998, -0.562433681, 0.016354000000000035, 0.863548, -0.787318, 0.14728599999999997, 2.6399543999999997, -1.7218171, 0.014062999999999881, -1.2874889999999999, 1.9438699999999998]
2.5268430000000004


In [ ]:
#Query = np.array(query) 
#Sentence = np.array(sentence)

In [153]:
import csv
with open('sentence_QA_vector.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['query_id','query_dim0','query_dim1','query_dim2','query_dim3','query_dim4','query_dim5','query_dim6','query_dim7','query_dim8','query_dim9','query_dim10','query_dim11','query_dim12','query_dim13','query_dim14','query_dim15','query_dim16','query_dim17','query_dim18','query_dim19','query_dim20','query_dim21','query_dim22','query_dim23','query_dim24','query_dim25','query_dim26','query_dim27','query_dim28','query_dim29','query_dim30','query_dim31','query_dim32','query_dim33','query_dim34','query_dim35','query_dim36','query_dim37','query_dim38','query_dim39','query_dim40','query_dim41','query_dim42','query_dim43','query_dim44','query_dim45','query_dim46','query_dim47','query_dim48','query_dim49','main_category','sentence_id','sentence_dim0','sentence_dim1','sentence_dim2','sentence_dim3','sentence_dim4','sentence_dim5','sentence_dim6','sentence_dim7','sentence_dim8','sentence_dim9','sentence_dim10','sentence_dim11','sentence_dim12','sentence_dim13','sentence_dim14','sentence_dim15','sentence_dim16','sentence_dim17','sentence_dim18','sentence_dim19','sentence_dim20','sentence_dim21','sentence_dim22','sentence_dim23','sentence_dim24','sentence_dim25','sentence_dim26','sentence_dim27','sentence_dim28','sentence_dim29','sentence_dim30','sentence_dim31','sentence_dim32','sentence_dim33','sentence_dim34','sentence_dim35','sentence_dim36','sentence_dim37','sentence_dim38','sentence_dim39','sentence_dim40','sentence_dim41','sentence_dim42','sentence_dim43','sentence_dim44','sentence_dim45','sentence_dim46','sentence_dim47','sentence_dim48','sentence_dim49','whether_groundTruth'])
    for i in range(0,len(query_id)):
        writer.writerow([query_id[i],query[i][0],query[i][1],query[i][2],query[i][3],query[i][4],query[i][5],query[i][6],query[i][7],query[i][8],query[i][9],query[i][10],query[i][11],query[i][12],query[i][13],query[i][14],query[i][15],query[i][16],query[i][17],query[i][18],query[i][19],query[i][20],query[i][21],query[i][22],query[i][23],query[i][24],query[i][25],query[i][26],query[i][27],query[i][28],query[i][29],query[i][30],query[i][31],query[i][32],query[i][33],query[i][34],query[i][35],query[i][36],query[i][37],query[i][38],query[i][39],query[i][40],query[i][41],query[i][42],query[i][43],query[i][44],query[i][45],query[i][46],query[i][47],query[i][48],query[i][49],category[i],sentence_id[i],sentence[i][0],sentence[i][1],sentence[i][2],sentence[i][3],sentence[i][4],sentence[i][5],sentence[i][6],sentence[i][7],sentence[i][8],sentence[i][9],sentence[i][10],sentence[i][11],sentence[i][12],sentence[i][13],sentence[i][14],sentence[i][15],sentence[i][16],sentence[i][17],sentence[i][18],sentence[i][19],sentence[i][20],sentence[i][21],sentence[i][22],sentence[i][23],sentence[i][24],sentence[i][25],sentence[i][26],sentence[i][27],sentence[i][28],sentence[i][29],sentence[i][30],sentence[i][31],sentence[i][32],sentence[i][33],sentence[i][34],sentence[i][35],sentence[i][36],sentence[i][37],sentence[i][38],sentence[i][39],sentence[i][40],sentence[i][41],sentence[i][42],sentence[i][43],sentence[i][44],sentence[i][45],sentence[i][46],sentence[i][47],sentence[i][48],sentence[i][49],whether_groundTruth[i]])
    

In [8]:
import pandas as pd
import numpy as np
sentence_vector_data = pd.read_csv('sentence_QA_vector.csv')
sentence_vector_data.head()

,query_id,query_dim0,query_dim1,query_dim2,query_dim3,query_dim4,query_dim5,query_dim6,query_dim7,query_dim8,...,sentence_dim41,sentence_dim42,sentence_dim43,sentence_dim44,sentence_dim45,sentence_dim46,sentence_dim47,sentence_dim48,sentence_dim49,whether_groundTruth
0,0,2.526843,1.491834,3.135254,1.06661,3.604157,-2.187905,-0.34216,0.22392,-1.708821,...,4.296646,-1.247253,-3.063499,2.485940,5.341059,-15.323900,-0.907039,-5.777957,-12.966610,1
1,0,2.526843,1.491834,3.135254,1.06661,3.604157,-2.187905,-0.34216,0.22392,-1.708821,...,1.544746,5.874265,-7.135296,4.294561,5.767480,-11.559292,4.163268,-3.945968,-11.521741,1
2,0,2.526843,1.491834,3.135254,1.06661,3.604157,-2.187905,-0.34216,0.22392,-1.708821,...,-0.208306,-0.446918,-1.764933,-1.832892,0.818763,-0.267611,-0.016267,-0.911197,-1.375337,1
3,0,2.526843,1.491834,3.135254,1.06661,3.604157,-2.187905,-0.34216,0.22392,-1.708821,...,3.613724,1.293993,1.943949,0.361942,1.395195,-7.706854,-0.992035,-3.255798,-5.949074,1
4,0,2.526843,1.491834,3.135254,1.06661,3.604157,-2.187905,-0.34216,0.22392,-1.708821,...,2.217443,1.150270,-0.494607,-1.077803,2.025698,-3.443631,1.278419,-1.155921,-0.710945,0


In [9]:
#sentence_vector_data.tail()

### use SVM as a classifer to train
x: main category, query_vector, sentence_vector
y: whether_groundTruth
dataset: 80: train, 20: test
for test: use ROUGE to evaluate. 

### just use a small part of dataset to train

In [10]:
from sklearn import preprocessing
np.random.seed(12)
label_encoder = preprocessing.LabelEncoder()

sentence_vector_data["main_category"] = label_encoder.fit_transform(sentence_vector_data["main_category"])

In [25]:
len(sentence_vector_data)

1438099

In [17]:
#sentence_vector_data[5900:5950]

### use a small part of dataset

In [26]:
num_train = 50000
num_test = 60000

In [27]:
sentence_train = sentence_vector_data.iloc[:num_train]
sentence_test = sentence_vector_data.iloc[num_train:num_test]

In [28]:
y_train = sentence_train['whether_groundTruth'].values
X_train = sentence_train.drop(['query_id','sentence_id','whether_groundTruth'],axis = 1).values
X_test = sentence_test.drop(['query_id','sentence_id','whether_groundTruth'],axis = 1).values

In [29]:
svm = SVC()
print('***********************training************************')
svm.fit(X_train, y_train)

***********************training************************


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [30]:
print('***********************predict*************************')
prediction = svm.predict(X_test)

***********************predict*************************


## Rouge evaluation

### a problem in SVM model: the predict result for most sentences are "not contained in answer". even there are sometimes for a query, there are no sentences predicted in the answer...

In [45]:
prediction[170:200]

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [105]:
rouge_1_F = []
rouge_1_P = []
rouge_1_R = []
rouge_2_F = []
rouge_2_P = []
rouge_2_R = []
rouge_l_F = []
rouge_l_P = []
rouge_l_R = []

train_num = 50000
test_num = 60000

question_flag = sentence_data['query_id'][train_num]
sentence_predicted = ''

for i in range(0,len(prediction)):
    if prediction[i] == 1:
        
        sentence_predicted_in_answer = sentence_data['sentence'][train_num+i]
        question_id = sentence_data['query_id'][train_num+i]
        
        if question_id == question_flag:
            sentence_predicted = sentence_predicted + sentence_predicted_in_answer
        else:
            #for this question_flag: 
            hypothesis = sentence_predicted
            reference = bestAnswers[question_flag]
            rouge = Rouge()
            scores = rouge.get_scores(reference, hypothesis)
            rouge_1_f = scores[0]['rouge-1']['f']
            rouge_1_p = scores[0]['rouge-1']['p']
            rouge_1_r = scores[0]['rouge-1']['r']
            rouge_2_f = scores[0]['rouge-2']['f']
            rouge_2_p = scores[0]['rouge-2']['p']
            rouge_2_r = scores[0]['rouge-2']['r']
            rouge_l_f = scores[0]['rouge-l']['f']
            rouge_l_p = scores[0]['rouge-l']['p']
            rouge_l_r = scores[0]['rouge-l']['r']
            
            rouge_1_F.append(rouge_1_f)
            rouge_1_P.append(rouge_1_p)
            rouge_1_R.append(rouge_1_r)
            rouge_2_F.append(rouge_2_f)
            rouge_2_P.append(rouge_2_p)
            rouge_2_R.append(rouge_2_r)
            rouge_l_F.append(rouge_l_f)
            rouge_l_P.append(rouge_l_p)
            rouge_l_R.append(rouge_l_r)
            
            #update question_flag:
            question_flag = sentence_data['query_id'][train_num+i]
            #update sentence_predicted:
            sentence_predicted = ''
            sentence_predicted = sentence_predicted + sentence_predicted_in_answer

In [106]:
sum = 0
for i in range(0,len(rouge_1_F)):
    sum = sum + rouge_1_F[i]
_rouge_1_f = sum/len(rouge_1_F)
print("rouge_1: f: ")
print(_rouge_1_f)
sum = 0
for i in range(0,len(rouge_1_P)):
    sum = sum + rouge_1_P[i]
_rouge_1_p = sum/len(rouge_1_P)
print("rouge_1: p: ")
print(_rouge_1_p)
sum = 0
for i in range(0,len(rouge_1_R)):
    sum = sum + rouge_1_R[i]
_rouge_1_r = sum/len(rouge_1_R)
print("rouge_1: Rr: ")
print(_rouge_1_r)

sum = 0
for i in range(0,len(rouge_2_F)):
    sum = sum + rouge_2_F[i]
_rouge_2_f = sum/len(rouge_2_F)
print("rouge_2: f: ")
print(_rouge_2_f)
sum = 0
for i in range(0,len(rouge_2_P)):
    sum = sum + rouge_2_P[i]
_rouge_2_p = sum/len(rouge_2_P)
print("rouge_2: p: ")
print(_rouge_2_p)
sum = 0
for i in range(0,len(rouge_2_R)):
    sum = sum + rouge_2_R[i]
_rouge_2_r = sum/len(rouge_2_R)
print("rouge_2: Rr: ")
print(_rouge_2_r)

sum = 0
for i in range(0,len(rouge_l_F)):
    sum = sum + rouge_l_F[i]
_rouge_l_f = sum/len(rouge_l_F)
print("rouge_L: f: ")
print(_rouge_l_f)
sum = 0
for i in range(0,len(rouge_l_P)):
    sum = sum + rouge_l_P[i]
_rouge_l_p = sum/len(rouge_l_P)
print("rouge_L: p: ")
print(_rouge_l_p)
sum = 0
for i in range(0,len(rouge_l_R)):
    sum = sum + rouge_l_R[i]
_rouge_l_r = sum/len(rouge_l_R)
print("rouge_L: Rr: ")
print(_rouge_l_r)

rouge_1: f: 
0.5438155787113726
rouge_1: p: 
0.5180162711084332
rouge_1: Rr: 
0.711356760051562
rouge_2: f: 
0.49016099411627007
rouge_2: p: 
0.46842395444796503
rouge_2: Rr: 
0.6016974835641854
rouge_L: f: 
0.4942657575598574
rouge_L: p: 
0.5004499786698657
rouge_L: Rr: 
0.6995877675898268


## LexRank
used A python package for Document Summarization models:         
sumy 0.6.0 from: Michal Belica

In [7]:
import json
mydata = json.load(open('/Users/zhangyiman/Desktop/nfL6.json','r'))
questions = []
bestAnswers = []
NotBestAnswers = []
main_categorys = []
q_id = []
for q_a in mydata:
    questions.append(q_a['question'])
    bestAnswers.append(q_a['answer'])
    NotBestAnswers.append(q_a['nbestanswers'])
    main_categorys.append(q_a['main_category'])
    q_id.append(q_a['id'])
    
# sentences candidate

query_id = []
query = []
category = []
sentence_id = []
sentence = []
whether_groundTruth = []
for j in range (0,len(NotBestAnswers)):
    # for every question, 
    sentence_candidate = []
    Ground_Truth_sentence = []
    
    # there are len_sen answers for this question
    len_sen = len(NotBestAnswers[j])
    for a in range(0,len_sen):
        str = NotBestAnswers[j][a]
        str_split = str.split('. ')
        # reduce if sentence is null
        for i in range(0,len(str_split)):
            if len(str_split[i])>=2:
                sentence_candidate.append(str_split[i])
                
    #Ground Truth sentences
    str = bestAnswers[j]
    str_split = str.split('. ')
    for i in range(0,len(str_split)):
        Ground_Truth_sentence.append(str_split[i])
        
    #Whether ground truth
    for i in range(0,len(sentence_candidate)):
        if sentence_candidate[i] not in Ground_Truth_sentence:
            Ground_Truth = 0
        else:
            Ground_Truth = 1
        query_id.append(j)
        query.append(questions[j])
        category.append(main_categorys[j])
        sentence_id.append(i)
        sentence.append(sentence_candidate[i])
        whether_groundTruth.append(Ground_Truth)
      

In [8]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [9]:
from rouge import Rouge

In [10]:
LANGUAGE = "english"
SENTENCES_COUNT = 3

In [11]:
## For each qustion: add sentences candidates in a txt file. 
## Then use the lexRank to summarize this document.
## calculate the rouge, for each question. and use the average score as ths score of this model.

### use part of dataset to evaluate

In [14]:
len(query_id)

1438099

In [18]:
query_id[100020:100030]

[10125, 10125, 10125, 10125, 10125, 10125, 10125, 10126, 10126, 10126]

In [19]:
query_id = query_id[0:100026]

In [20]:
rouge_1_F = []
rouge_1_P = []
rouge_1_R = []
rouge_2_F = []
rouge_2_P = []
rouge_2_R = []
rouge_l_F = []
rouge_l_P = []
rouge_l_R = []

query_id_flag = 0

hypothesis = ' '

with open('document.txt','w') as f:
    f.write(' ')
f.close()

for i in range(0,len(query_id)):
    #for one query
    if query_id[i] == query_id_flag:
        with open('document.txt','a') as f:
            f.write(sentence[i])
            f.write('\n')
        f.close()
        
    else:
        #use the document to summarize answer and get rouge score    
        parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)   
      
        evaluated_sentences = (summarizer(parser.document, SENTENCES_COUNT))
        if len(evaluated_sentences)>0:
            for i in range(0,3):
                #evaluated_sentences[i].words
                str_111 = ' '
                for i in range(0,len(evaluated_sentences[0].words)):
                    str_111 = str_111 + evaluated_sentences[0].words[i]
                    str_111 = str_111 + ' '
                hypothesis = hypothesis + str_111

            reference = bestAnswers[query_id_flag]
            rouge = Rouge()
            scores = rouge.get_scores(reference, hypothesis)
            rouge_1_f = scores[0]['rouge-1']['f']
            rouge_1_p = scores[0]['rouge-1']['p']
            rouge_1_r = scores[0]['rouge-1']['r']
            rouge_2_f = scores[0]['rouge-2']['f']
            rouge_2_p = scores[0]['rouge-2']['p']
            rouge_2_r = scores[0]['rouge-2']['r']
            rouge_l_f = scores[0]['rouge-l']['f']
            rouge_l_p = scores[0]['rouge-l']['p']
            rouge_l_r = scores[0]['rouge-l']['r']
            
            rouge_1_F.append(rouge_1_f)
            rouge_1_P.append(rouge_1_p)
            rouge_1_R.append(rouge_1_r)
            rouge_2_F.append(rouge_2_f)
            rouge_2_P.append(rouge_2_p)
            rouge_2_R.append(rouge_2_r)
            rouge_l_F.append(rouge_l_f)
            rouge_l_P.append(rouge_l_p)
            rouge_l_R.append(rouge_l_r)

        #update query_id_flag
        query_id_flag = query_id_flag + 1
    
        #update document.txt
        with open('document.txt','w') as f:
            f.write(' ')
        f.close()
        
        #update hypothesis
        hypothesis = ' '


In [21]:
sum = 0
for i in range(0,len(rouge_1_F)):
    sum = sum + rouge_1_F[i]
_rouge_1_f = sum/len(rouge_1_F)
print("rouge_1: f: ")
print(_rouge_1_f)
sum = 0
for i in range(0,len(rouge_1_P)):
    sum = sum + rouge_1_P[i]
_rouge_1_p = sum/len(rouge_1_P)
print("rouge_1: p: ")
print(_rouge_1_p)
sum = 0
for i in range(0,len(rouge_1_R)):
    sum = sum + rouge_1_R[i]
_rouge_1_r = sum/len(rouge_1_R)
print("rouge_1: Rr: ")
print(_rouge_1_r)

sum = 0
for i in range(0,len(rouge_2_F)):
    sum = sum + rouge_2_F[i]
_rouge_2_f = sum/len(rouge_2_F)
print("rouge_2: f: ")
print(_rouge_2_f)
sum = 0
for i in range(0,len(rouge_2_P)):
    sum = sum + rouge_2_P[i]
_rouge_2_p = sum/len(rouge_2_P)
print("rouge_2: p: ")
print(_rouge_2_p)
sum = 0
for i in range(0,len(rouge_2_R)):
    sum = sum + rouge_2_R[i]
_rouge_2_r = sum/len(rouge_2_R)
print("rouge_2: Rr: ")
print(_rouge_2_r)

sum = 0
for i in range(0,len(rouge_l_F)):
    sum = sum + rouge_l_F[i]
_rouge_l_f = sum/len(rouge_l_F)
print("rouge_L: f: ")
print(_rouge_l_f)
sum = 0
for i in range(0,len(rouge_l_P)):
    sum = sum + rouge_l_P[i]
_rouge_l_p = sum/len(rouge_l_P)
print("rouge_L: p: ")
print(_rouge_l_p)
sum = 0
for i in range(0,len(rouge_l_R)):
    sum = sum + rouge_l_R[i]
_rouge_l_r = sum/len(rouge_l_R)
print("rouge_L: Rr: ")
print(_rouge_l_r)

rouge_1: f: 
0.5026268618931697
rouge_1: p: 
0.4443284853503644
rouge_1: Rr: 
0.6496826927942
rouge_2: f: 
0.38077093884293656
rouge_2: p: 
0.34336433758611445
rouge_2: Rr: 
0.4736010587935103
rouge_L: f: 
0.2471571755187573
rouge_L: p: 
0.4722198284253882
rouge_L: Rr: 
0.237431990835491


## Luhn - heurestic method 

In [42]:
import json
mydata = json.load(open('/Users/zhangyiman/Desktop/nfL6.json','r'))
questions = []
bestAnswers = []
NotBestAnswers = []
main_categorys = []
q_id = []
for q_a in mydata:
    questions.append(q_a['question'])
    bestAnswers.append(q_a['answer'])
    NotBestAnswers.append(q_a['nbestanswers'])
    main_categorys.append(q_a['main_category'])
    q_id.append(q_a['id'])
    
# sentences candidate

query_id = []
query = []
category = []
sentence_id = []
sentence = []
whether_groundTruth = []
for j in range (0,len(NotBestAnswers)):
    # for every question, 
    sentence_candidate = []
    Ground_Truth_sentence = []
    
    # there are len_sen answers for this question
    len_sen = len(NotBestAnswers[j])
    for a in range(0,len_sen):
        str = NotBestAnswers[j][a]
        str_split = str.split('. ')
        # reduce if sentence is null
        for i in range(0,len(str_split)):
            if len(str_split[i])>=2:
                sentence_candidate.append(str_split[i])
                
    #Ground Truth sentences
    str = bestAnswers[j]
    str_split = str.split('. ')
    for i in range(0,len(str_split)):
        Ground_Truth_sentence.append(str_split[i])
        
    #Whether ground truth
    for i in range(0,len(sentence_candidate)):
        if sentence_candidate[i] not in Ground_Truth_sentence:
            Ground_Truth = 0
        else:
            Ground_Truth = 1
        query_id.append(j)
        query.append(questions[j])
        category.append(main_categorys[j])
        sentence_id.append(i)
        sentence.append(sentence_candidate[i])
        whether_groundTruth.append(Ground_Truth)
      

In [43]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [44]:
from rouge import Rouge

### use part of dataset

In [45]:
query_id = query_id[0:100026]

In [46]:
LANGUAGE = "english"
SENTENCES_COUNT = 3

rouge_1_F = []
rouge_1_P = []
rouge_1_R = []
rouge_2_F = []
rouge_2_P = []
rouge_2_R = []
rouge_l_F = []
rouge_l_P = []
rouge_l_R = []

query_id_flag = 0

hypothesis = ' '

with open('document.txt','w') as f:
    f.write(' ')
f.close()

for i in range(0,len(query_id)):
    #for one query
    if query_id[i] == query_id_flag:
        with open('document.txt','a') as f:
            f.write(sentence[i])
            f.write('\n')
        f.close()
        
    else:
        #use the document to summarize answer and get rouge score    
        parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)   
      
        evaluated_sentences = (summarizer(parser.document, SENTENCES_COUNT))
        if len(evaluated_sentences)>0:
            for i in range(0,3):
                #evaluated_sentences[i].words
                str_111 = ' '
                for i in range(0,len(evaluated_sentences[0].words)):
                    str_111 = str_111 + evaluated_sentences[0].words[i]
                    str_111 = str_111 + ' '
                hypothesis = hypothesis + str_111

            reference = bestAnswers[query_id_flag]
            rouge = Rouge()
            scores = rouge.get_scores(reference, hypothesis)
            rouge_1_f = scores[0]['rouge-1']['f']
            rouge_1_p = scores[0]['rouge-1']['p']
            rouge_1_r = scores[0]['rouge-1']['r']
            rouge_2_f = scores[0]['rouge-2']['f']
            rouge_2_p = scores[0]['rouge-2']['p']
            rouge_2_r = scores[0]['rouge-2']['r']
            rouge_l_f = scores[0]['rouge-l']['f']
            rouge_l_p = scores[0]['rouge-l']['p']
            rouge_l_r = scores[0]['rouge-l']['r']
            
            rouge_1_F.append(rouge_1_f)
            rouge_1_P.append(rouge_1_p)
            rouge_1_R.append(rouge_1_r)
            rouge_2_F.append(rouge_2_f)
            rouge_2_P.append(rouge_2_p)
            rouge_2_R.append(rouge_2_r)
            rouge_l_F.append(rouge_l_f)
            rouge_l_P.append(rouge_l_p)
            rouge_l_R.append(rouge_l_r)

        #update query_id_flag
        query_id_flag = query_id_flag + 1
    
        #update document.txt
        with open('document.txt','w') as f:
            f.write(' ')
        f.close()
        
        #update hypothesis
        hypothesis = ' '



In [47]:
sum = 0
for i in range(0,len(rouge_1_F)):
    sum = sum + rouge_1_F[i]
_rouge_1_f = sum/len(rouge_1_F)
print("rouge_1: f: ")
print(_rouge_1_f)
sum = 0
for i in range(0,len(rouge_1_P)):
    sum = sum + rouge_1_P[i]
_rouge_1_p = sum/len(rouge_1_P)
print("rouge_1: p: ")
print(_rouge_1_p)
sum = 0
for i in range(0,len(rouge_1_R)):
    sum = sum + rouge_1_R[i]
_rouge_1_r = sum/len(rouge_1_R)
print("rouge_1: Rr: ")
print(_rouge_1_r)

sum = 0
for i in range(0,len(rouge_2_F)):
    sum = sum + rouge_2_F[i]
_rouge_2_f = sum/len(rouge_2_F)
print("rouge_2: f: ")
print(_rouge_2_f)
sum = 0
for i in range(0,len(rouge_2_P)):
    sum = sum + rouge_2_P[i]
_rouge_2_p = sum/len(rouge_2_P)
print("rouge_2: p: ")
print(_rouge_2_p)
sum = 0
for i in range(0,len(rouge_2_R)):
    sum = sum + rouge_2_R[i]
_rouge_2_r = sum/len(rouge_2_R)
print("rouge_2: Rr: ")
print(_rouge_2_r)

sum = 0
for i in range(0,len(rouge_l_F)):
    sum = sum + rouge_l_F[i]
_rouge_l_f = sum/len(rouge_l_F)
print("rouge_L: f: ")
print(_rouge_l_f)
sum = 0
for i in range(0,len(rouge_l_P)):
    sum = sum + rouge_l_P[i]
_rouge_l_p = sum/len(rouge_l_P)
print("rouge_L: p: ")
print(_rouge_l_p)
sum = 0
for i in range(0,len(rouge_l_R)):
    sum = sum + rouge_l_R[i]
_rouge_l_r = sum/len(rouge_l_R)
print("rouge_L: Rr: ")
print(_rouge_l_r)

rouge_1: f: 
0.48250960513661045
rouge_1: p: 
0.43236977950887484
rouge_1: Rr: 
0.6159745300853949
rouge_2: f: 
0.35864907716308947
rouge_2: p: 
0.325525595778387
rouge_2: Rr: 
0.44290731827035873
rouge_L: f: 
0.23445843433946761
rouge_L: p: 
0.45938242815423447
rouge_L: Rr: 
0.22457448651521883


In [48]:
## todo: CNN

In [49]:
## todo: another dataset

# dataset_2

In [1]:
import re
import fileinput
lines = []
for line in fileinput.input("QA_dataset.txt"):
    lines.append(line)

In [2]:
db_query_id = []
db_query = []
db_answer = []
db_score = []

In [3]:
# score for answer: 4 : excellent 3: good

In [4]:
question_id = -1

for i in range(0,len(lines)):
    x = lines[i].split('\t',1)
    
    if x[0] == 'QUESTION':
        y = x[1].split('\t',2)
        question_id = question_id + 1
        question = y[2]
    else: 
        y = x[1].split('\t',2)
        score = int(y[1])
        answer = y[2]
        
        db_query_id.append(question_id)
        db_query.append(question)
        db_answer.append(answer)
        db_score.append(score)

In [5]:
# sentences candidate

query_id = []
query = []
sentence_id = []
sentence = []
score = []
# or whether_groundTruth

query_flag = 0
sen_ID = 0

for i in range(0,len(db_answer)):
    if db_query_id[i] == query_flag: 
        str = db_answer[i]
        str_split = str.split('. ')
        for j in range(0,len(str_split)):
            sentence.append(str_split[j])
            query_id.append(db_query_id[i])
            query.append(db_query[i])
            score.append(db_score[i])
            sentence_id.append(sen_ID)
            sen_ID = sen_ID + 1
    else: 
        query_flag = query_flag + 1
        sen_ID = 0
        str = db_answer[i]
        str_split = str.split('. ')
        for j in range(0,len(str_split)):
            sentence.append(str_split[j])
            query_id.append(db_query_id[i])
            query.append(db_query[i])
            score.append(db_score[i])
            sentence_id.append(sen_ID)
            sen_ID = sen_ID + 1


In [6]:
query_flag = 0
num_3 = 0
num_4 = 0
num_num_3 = []
num_num_4 = []

for i in range(0,len(sentence)):
    if query_id[i] == query_flag:
        if score[i] == 4: 
            num_4 = num_4 + 1
        if score[i] == 3: 
            num_3 = num_3 + 1
    else: 
        num_num_3.append(num_3)
        num_num_4.append(num_4)
        
        query_flag = query_flag + 1
        
        num_3 = 0
        num_4 = 0
        if score[i] == 4: 
            num_4 = num_4 + 1
        if score[i] == 3: 
            num_3 = num_3 + 1

# need to delete the query_id = 1014 in the dataframe because here we simply set num_num_3 and num_num_4 as 0
num_num_4.append(0)
num_num_3.append(0)

In [7]:
delete_question_id = []
for i in range(0,len(num_num_3)):
    if num_num_3[i] == 0 and num_num_4[i] == 0:
        delete_question_id.append(i)

In [8]:
delete_question_id[0:5]

[17, 56, 64, 67, 91]

In [9]:
delete_question_id.append(1014)

In [10]:
# set whether_select_this_sentence
# whether_select --> 0: not select 1:select
# for questions which have score 4 sentences, select these score 4 sentences
# for question which don't have score 4 sentences, select score 3 sentences. 

In [11]:
whether_select = []
for i in range(0,len(query_id)):
    if num_num_4[query_id[i]] != 0 :
        #set score 4 sentences as selected
        if score[i] == 4:
            whether_select.append(1)
        else:
            whether_select.append(0)
    else: 
        # set score 3 sentences as selected
        if score[i] ==3:
            whether_select.append(1)
        else:
            whether_select.append(0)   

In [12]:
#len(query_id)
#len(whether_select)
len(sentence)

153335

### set a list: Best_Answer[] --> use to evaluate the baselines 

In [13]:
best_answer = []
str = ''
query_flag = 0

for i in range(0,len(whether_select)):
    if query_id[i] == query_flag:
        if whether_select[i] == 1:
            str = str + sentence[i]
    else: 
        best_answer.append(str)
        query_flag = query_flag + 1
        str = ''

In [14]:
len(best_answer)

1014

In [15]:
best_answer[1]

"Here is what I do...I soap it up real good when showering and rinse thoroughly, when I am drying off, I use qtips to dry out the belly button and remove lint, etcafterwards I spray a small amount of body spray in there to keep it smelling freshDo this daily and you're gooI know exactly how you feel, I live near the beach and every time I go, I always find sand hidden deep inside my bellyholeIf your at the beach and you get sand all over your belly, you may have to just brush it off unless you want to discretely clean your navel with your fingerAfter you get home, you can do a more thorough cleaning by using a qtipGood luck, beach sand is awfully stickMy.boyfriend absolutely hates touching his belly buttonHe keeps it clean but it grosses him out everytimeAnd It's not even a deep belly button you wipe the cloth across his stomach and It's clean, hes a stick so to sayI don't have issues with my own belly button th.\nSeriously, and I am not being sarcastic hereHe needs to wash his belly b

### write into csv and read as dataframe

In [16]:
import csv

with open('sentence_QA_2.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['q_id','query_id','query','sentence_id','sentence','score','whether_select'])
    for i in range(0,len(query_id)):
        writer.writerow([query_id[i],query_id[i],query[i],sentence_id[i],sentence[i],score[i],whether_select[i]])
    

In [17]:
import pandas as pd
import numpy as np
sentence_df = pd.read_csv('sentence_QA_2.csv')
#sentence_df[400:450]

In [18]:
sentence_df.head()

,q_id,query_id,query,sentence_id,sentence,score,whether_select
0,0,0,One side of my body tenses/freezes? Help?\tI'm...,0,Don't seek help here...Go and see a doctor imm...,4,1
1,0,0,One side of my body tenses/freezes? Help?\tI'm...,1,"I just did a body scan and my right side, ..",2,0
2,0,0,One side of my body tenses/freezes? Help?\tI'm...,2,My entire body is one giant tense muscle,2,0
3,0,0,One side of my body tenses/freezes? Help?\tI'm...,3,If I tense one eye ..,2,0
4,0,0,One side of my body tenses/freezes? Help?\tI'm...,4,Tense neck muscle when i yawn? The throbbing i...,2,0


#### remove query which don't have ground good answer sentences and use less in experiment. 

In [19]:
sentence_df = sentence_df.set_index('q_id')

In [20]:
for i in range(0,len(delete_question_id)):
    sentence_df = sentence_df.drop(delete_question_id[i],axis = 0)

In [21]:
#sentence_df[156:200]

In [22]:
sentence_df = sentence_df.dropna(axis=0,how='any') #drop all rows that have any NaN values

## Random select sentence

#### average best_answer_length

In [201]:
sum_s = 0
num_s = 0
for i in range(0,len(best_answer)):
    str = best_answer[i]
    str_split = str.split('.')
    j = 0
    while j < len(str_split):
        if str_split[j] == '':
            del str_split[j]
            j = j -1
        j = j + 1
    sum_s = sum_s + len(str_split)
    num_s = num_s + 1
average_length = int(sum_s/num_s)
print(average_length)

8


In [23]:
import random
from collections import defaultdict
from rouge import Rouge

In [203]:
rouge_1_f_sum = 0
rouge_1_p_sum = 0
rouge_1_r_sum = 0
rouge_2_f_sum = 0
rouge_2_p_sum = 0
rouge_2_r_sum = 0
rouge_l_f_sum = 0
rouge_l_p_sum = 0
rouge_l_r_sum = 0
num = 0

for i in range(0,1014):
    if len(sentence_df[sentence_df['query_id']== i]) > 8: 
        sen_num = random.sample(range(len(sentence_df[sentence_df['query_id']== i])),8)
        hypothesis = ''
        for j in range(0,len(sen_num)):
            sub = sentence_df[sentence_df['query_id']==i]
            sub = sub.set_index('sentence_id')
            
            sentence_sel = sub['sentence'].values[j]
            hypothesis = hypothesis + sentence_sel
            
        reference = best_answer[i]
        rouge = Rouge()
        scores = rouge.get_scores(reference, hypothesis)
        rouge_1_f = scores[0]['rouge-1']['f']
        rouge_1_p = scores[0]['rouge-1']['p']
        rouge_1_r = scores[0]['rouge-1']['r']
        rouge_2_f = scores[0]['rouge-2']['f']
        rouge_2_p = scores[0]['rouge-2']['p']
        rouge_2_r = scores[0]['rouge-2']['r']
        rouge_l_f = scores[0]['rouge-l']['f']
        rouge_l_p = scores[0]['rouge-l']['p']
        rouge_l_r = scores[0]['rouge-l']['r']
    
        rouge_1_f_sum = rouge_1_f_sum + rouge_1_f
        rouge_1_p_sum = rouge_1_p_sum + rouge_1_p
        rouge_1_r_sum = rouge_1_r_sum + rouge_1_r
        rouge_2_f_sum = rouge_2_f_sum + rouge_2_f
        rouge_2_p_sum = rouge_2_p_sum + rouge_2_p
        rouge_2_r_sum = rouge_2_r_sum + rouge_2_r
        rouge_l_f_sum = rouge_l_f_sum + rouge_l_f
        rouge_l_p_sum = rouge_l_p_sum + rouge_l_p
        rouge_l_r_sum = rouge_l_r_sum + rouge_l_r
    
        num = num + 1
        
rouge_1_F = rouge_1_f_sum/num
rouge_1_P = rouge_1_p_sum/num
rouge_1_R = rouge_1_r_sum/num
rouge_2_F = rouge_2_f_sum/num
rouge_2_P = rouge_2_p_sum/num
rouge_2_R = rouge_2_r_sum/num
rouge_l_F = rouge_l_f_sum/num
rouge_l_P = rouge_l_p_sum/num
rouge_l_R = rouge_l_r_sum/num

In [204]:
print("rouge_1: f: ")
print(rouge_1_F)
print("rouge_1: p: ")
print(rouge_1_P)
print("rouge_1: r: ")
print(rouge_1_R)

print("rouge_2: f: ")
print(rouge_2_F)
print("rouge_2: p: ")
print(rouge_2_P)
print("rouge_2: r: ")
print(rouge_2_R)
      
print("rouge_L: f: ")
print(rouge_l_F)
print("rouge_L: p: ")
print(rouge_l_P)
print("rouge_L: r: ")
print(rouge_l_R)

rouge_1: f: 
0.22430505028259645
rouge_1: p: 
0.21549287395592748
rouge_1: r: 
0.3627529454799493
rouge_2: f: 
0.10173984826122628
rouge_2: p: 
0.08984945021209327
rouge_2: r: 
0.2001187128239444
rouge_L: f: 
0.11317252993971307
rouge_L: p: 
0.14225121541177846
rouge_L: r: 
0.28954199841489353


## SVM for second dataset

In [24]:
import numpy as np
import csv
#import datetime
from sklearn.svm import SVC
#import os
import pandas as pd
from sklearn import metrics
#cwd = os.getcwd()

In [25]:
f = open('/Users/zhangyiman/Desktop/word2vec_pre/glove.6B.50d.txt')
embeddings_index = {}
count = 0

for line in f:
        values_int = line.split('\t')
        #print(values_int[0])
        #values = values_int[0].split(' ',1)
        values = values_int[0].split(' ')
        id = values[0]
        #print(id)
        #coefs = np.asarray(values[1:])
        coefs = values[1:]
        #print(coefs)
        embeddings_index[id] = coefs
f.close()

In [26]:
import re
def sentence_vector(sentence_str):
    str = sentence_str
    #str = str.replace(',',' ,')
    str = str.replace("'s",'')
    str = str.replace("determent",'deter')
    str = str.replace("200B",'200 b')
    token = re.compile("[\w]+(?=n't)|n't|\'m|\'ll|[\w]+|[.?!;,\-\(\)—\:']")
    tokens = token.findall(str)
    
    #str_convert = ''.join(tokens[0])
    a = [0]*50
    for i in range(0,len(tokens)):
            for j in range(0,50):
                str_convert = ''.join(tokens[i])
                str_lower = str_convert.lower()
                try: 
                    m = float(embeddings_index[str_lower][j])
                except:
                    m = 0
                a[j] = a[j] + m
    for i in range(0,50):
        if len(tokens) ==0:
            a[i] = 0
        else:
            a[i] = a[i]/len(tokens)
    return(a)

In [27]:
for i in range(0,len(query)):
    str = query[i]
    query[i] = sentence_vector(str)
    str = sentence[i]
    sentence[i] = sentence_vector(str)

In [28]:
query[1]

[0.2567643953608248,
 0.12853462371134036,
 -0.0035550721649484575,
 -0.2723664639175257,
 0.4016127061855672,
 0.11410566185567009,
 -0.4018567907216495,
 0.029143582474226826,
 -0.2166815389175259,
 -0.04793512340206185,
 -0.012361213402061854,
 0.13971055927835063,
 -0.44455645721649467,
 -0.1285194952061856,
 0.5957187108247424,
 0.30770805000000007,
 0.041629103608247425,
 -0.0048980278350515055,
 -0.3024270948453608,
 -0.4270346170103095,
 -0.10960781443298959,
 0.2769147041237113,
 0.35894074226804135,
 0.0443243298969072,
 0.3005610345360826,
 -1.6955116340206187,
 -0.357974675257732,
 0.1939805773195878,
 0.4037584721649484,
 -0.4255267628865982,
 3.2488940618556694,
 0.3189583257731959,
 -0.2566510463917528,
 -0.18459720783505154,
 0.03316788324742269,
 -0.07661612525773198,
 0.15693918711340207,
 0.22201750824742286,
 0.2559271011340207,
 -0.18962159793814432,
 -0.23580749312886604,
 0.07441659015463911,
 0.004442796237113394,
 0.23278254783505162,
 -0.011192520618556705,
 0

In [29]:
print(query[1])
print(query[1][0])

[0.2567643953608248, 0.12853462371134036, -0.0035550721649484575, -0.2723664639175257, 0.4016127061855672, 0.11410566185567009, -0.4018567907216495, 0.029143582474226826, -0.2166815389175259, -0.04793512340206185, -0.012361213402061854, 0.13971055927835063, -0.44455645721649467, -0.1285194952061856, 0.5957187108247424, 0.30770805000000007, 0.041629103608247425, -0.0048980278350515055, -0.3024270948453608, -0.4270346170103095, -0.10960781443298959, 0.2769147041237113, 0.35894074226804135, 0.0443243298969072, 0.3005610345360826, -1.6955116340206187, -0.357974675257732, 0.1939805773195878, 0.4037584721649484, -0.4255267628865982, 3.2488940618556694, 0.3189583257731959, -0.2566510463917528, -0.18459720783505154, 0.03316788324742269, -0.07661612525773198, 0.15693918711340207, 0.22201750824742286, 0.2559271011340207, -0.18962159793814432, -0.23580749312886604, 0.07441659015463911, 0.004442796237113394, 0.23278254783505162, -0.011192520618556705, 0.03726610567010306, -0.04595910221649484, -0.

In [30]:
import csv
with open('sentence_QA_2_vector.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['query_id','query_dim0','query_dim1','query_dim2','query_dim3','query_dim4','query_dim5','query_dim6','query_dim7','query_dim8','query_dim9','query_dim10','query_dim11','query_dim12','query_dim13','query_dim14','query_dim15','query_dim16','query_dim17','query_dim18','query_dim19','query_dim20','query_dim21','query_dim22','query_dim23','query_dim24','query_dim25','query_dim26','query_dim27','query_dim28','query_dim29','query_dim30','query_dim31','query_dim32','query_dim33','query_dim34','query_dim35','query_dim36','query_dim37','query_dim38','query_dim39','query_dim40','query_dim41','query_dim42','query_dim43','query_dim44','query_dim45','query_dim46','query_dim47','query_dim48','query_dim49','sentence_id','sentence_dim0','sentence_dim1','sentence_dim2','sentence_dim3','sentence_dim4','sentence_dim5','sentence_dim6','sentence_dim7','sentence_dim8','sentence_dim9','sentence_dim10','sentence_dim11','sentence_dim12','sentence_dim13','sentence_dim14','sentence_dim15','sentence_dim16','sentence_dim17','sentence_dim18','sentence_dim19','sentence_dim20','sentence_dim21','sentence_dim22','sentence_dim23','sentence_dim24','sentence_dim25','sentence_dim26','sentence_dim27','sentence_dim28','sentence_dim29','sentence_dim30','sentence_dim31','sentence_dim32','sentence_dim33','sentence_dim34','sentence_dim35','sentence_dim36','sentence_dim37','sentence_dim38','sentence_dim39','sentence_dim40','sentence_dim41','sentence_dim42','sentence_dim43','sentence_dim44','sentence_dim45','sentence_dim46','sentence_dim47','sentence_dim48','sentence_dim49','whether_select'])
    for i in range(0,len(query_id)):
        writer.writerow([query_id[i],query[i][0],query[i][1],query[i][2],query[i][3],query[i][4],query[i][5],query[i][6],query[i][7],query[i][8],query[i][9],query[i][10],query[i][11],query[i][12],query[i][13],query[i][14],query[i][15],query[i][16],query[i][17],query[i][18],query[i][19],query[i][20],query[i][21],query[i][22],query[i][23],query[i][24],query[i][25],query[i][26],query[i][27],query[i][28],query[i][29],query[i][30],query[i][31],query[i][32],query[i][33],query[i][34],query[i][35],query[i][36],query[i][37],query[i][38],query[i][39],query[i][40],query[i][41],query[i][42],query[i][43],query[i][44],query[i][45],query[i][46],query[i][47],query[i][48],query[i][49],sentence_id[i],sentence[i][0],sentence[i][1],sentence[i][2],sentence[i][3],sentence[i][4],sentence[i][5],sentence[i][6],sentence[i][7],sentence[i][8],sentence[i][9],sentence[i][10],sentence[i][11],sentence[i][12],sentence[i][13],sentence[i][14],sentence[i][15],sentence[i][16],sentence[i][17],sentence[i][18],sentence[i][19],sentence[i][20],sentence[i][21],sentence[i][22],sentence[i][23],sentence[i][24],sentence[i][25],sentence[i][26],sentence[i][27],sentence[i][28],sentence[i][29],sentence[i][30],sentence[i][31],sentence[i][32],sentence[i][33],sentence[i][34],sentence[i][35],sentence[i][36],sentence[i][37],sentence[i][38],sentence[i][39],sentence[i][40],sentence[i][41],sentence[i][42],sentence[i][43],sentence[i][44],sentence[i][45],sentence[i][46],sentence[i][47],sentence[i][48],sentence[i][49],whether_select[i]])
    

In [44]:
import pandas as pd
import numpy as np
sentence_vector_data = pd.read_csv('sentence_QA_2_vector.csv')
sentence_vector_data.head()

,query_id,query_dim0,query_dim1,query_dim2,query_dim3,query_dim4,query_dim5,query_dim6,query_dim7,query_dim8,...,sentence_dim41,sentence_dim42,sentence_dim43,sentence_dim44,sentence_dim45,sentence_dim46,sentence_dim47,sentence_dim48,sentence_dim49,whether_select
0,0,0.256764,0.128535,-0.003555,-0.272366,0.401613,0.114106,-0.401857,0.029144,-0.216682,...,0.131674,0.064359,0.050794,0.176399,0.102187,-0.079441,-0.163512,0.083254,0.242382,1
1,0,0.256764,0.128535,-0.003555,-0.272366,0.401613,0.114106,-0.401857,0.029144,-0.216682,...,0.140784,-0.141418,0.186607,-0.058426,0.068691,0.048071,-0.325942,0.096276,0.038044,0
2,0,0.256764,0.128535,-0.003555,-0.272366,0.401613,0.114106,-0.401857,0.029144,-0.216682,...,-0.107614,-0.268862,0.170626,0.058393,0.088156,0.143237,-0.276487,-0.201549,-0.184999,0
3,0,0.256764,0.128535,-0.003555,-0.272366,0.401613,0.114106,-0.401857,0.029144,-0.216682,...,0.254807,-0.047262,0.227747,0.000003,0.056698,0.081251,-0.170346,-0.051025,0.187470,0
4,0,0.256764,0.128535,-0.003555,-0.272366,0.401613,0.114106,-0.401857,0.029144,-0.216682,...,0.064894,-0.170583,0.172435,0.047038,0.097803,0.034851,-0.259232,-0.182091,-0.057514,0


In [45]:
0.8*len(sentence_vector_data)

122668.0

In [46]:
## use small part of dataset to train
num_train = 122668
#num_test = 60000

In [47]:
sentence_train = sentence_vector_data.iloc[:num_train]
#sentence_test = sentence_vector_data.iloc[num_train:num_test]
sentence_test = sentence_vector_data.iloc[num_train:]

In [49]:
y_train = sentence_train['whether_select'].values
X_train = sentence_train.drop('whether_select',axis = 1).values
X_test = sentence_test.drop('whether_select',axis = 1).values

In [50]:
svm = SVC()
print('***********************training************************')
svm.fit(X_train, y_train)

***********************training************************


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [51]:
print('***********************predict*************************')
prediction = svm.predict(X_test)

***********************predict*************************


### rouge evaluation for dataset 2 use baseline SVM

In [52]:
prediction[170:250]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
'''rouge_1_F = []
rouge_1_P = []
rouge_1_R = []
rouge_2_F = []
rouge_2_P = []
rouge_2_R = []
rouge_l_F = []
rouge_l_P = []
rouge_l_R = []

train_num = 50000
test_num = 60000

question_flag = sentence_data['query_id'][train_num]
sentence_predicted = ''

for i in range(0,len(prediction)):
    if prediction[i] == 1:
        
        sentence_predicted_in_answer = sentence_data['sentence'][train_num+i]
        question_id = sentence_data['query_id'][train_num+i]
        
        if question_id == question_flag:
            sentence_predicted = sentence_predicted + sentence_predicted_in_answer
        else:
            #for this question_flag: 
            hypothesis = sentence_predicted
            reference = bestAnswers[question_flag]
            rouge = Rouge()
            scores = rouge.get_scores(reference, hypothesis)
            rouge_1_f = scores[0]['rouge-1']['f']
            rouge_1_p = scores[0]['rouge-1']['p']
            rouge_1_r = scores[0]['rouge-1']['r']
            rouge_2_f = scores[0]['rouge-2']['f']
            rouge_2_p = scores[0]['rouge-2']['p']
            rouge_2_r = scores[0]['rouge-2']['r']
            rouge_l_f = scores[0]['rouge-l']['f']
            rouge_l_p = scores[0]['rouge-l']['p']
            rouge_l_r = scores[0]['rouge-l']['r']
            
            rouge_1_F.append(rouge_1_f)
            rouge_1_P.append(rouge_1_p)
            rouge_1_R.append(rouge_1_r)
            rouge_2_F.append(rouge_2_f)
            rouge_2_P.append(rouge_2_p)
            rouge_2_R.append(rouge_2_r)
            rouge_l_F.append(rouge_l_f)
            rouge_l_P.append(rouge_l_p)
            rouge_l_R.append(rouge_l_r)
            
            #update question_flag:
            question_flag = sentence_data['query_id'][train_num+i]
            #update sentence_predicted:
            sentence_predicted = ''
            sentence_predicted = sentence_predicted + sentence_predicted_in_answer'''

In [ ]:
'''sum = 0
for i in range(0,len(rouge_1_F)):
    sum = sum + rouge_1_F[i]
_rouge_1_f = sum/len(rouge_1_F)
print("rouge_1: f: ")
print(_rouge_1_f)
sum = 0
for i in range(0,len(rouge_1_P)):
    sum = sum + rouge_1_P[i]
_rouge_1_p = sum/len(rouge_1_P)
print("rouge_1: p: ")
print(_rouge_1_p)
sum = 0
for i in range(0,len(rouge_1_R)):
    sum = sum + rouge_1_R[i]
_rouge_1_r = sum/len(rouge_1_R)
print("rouge_1: Rr: ")
print(_rouge_1_r)

sum = 0
for i in range(0,len(rouge_2_F)):
    sum = sum + rouge_2_F[i]
_rouge_2_f = sum/len(rouge_2_F)
print("rouge_2: f: ")
print(_rouge_2_f)
sum = 0
for i in range(0,len(rouge_2_P)):
    sum = sum + rouge_2_P[i]
_rouge_2_p = sum/len(rouge_2_P)
print("rouge_2: p: ")
print(_rouge_2_p)
sum = 0
for i in range(0,len(rouge_2_R)):
    sum = sum + rouge_2_R[i]
_rouge_2_r = sum/len(rouge_2_R)
print("rouge_2: Rr: ")
print(_rouge_2_r)

sum = 0
for i in range(0,len(rouge_l_F)):
    sum = sum + rouge_l_F[i]
_rouge_l_f = sum/len(rouge_l_F)
print("rouge_L: f: ")
print(_rouge_l_f)
sum = 0
for i in range(0,len(rouge_l_P)):
    sum = sum + rouge_l_P[i]
_rouge_l_p = sum/len(rouge_l_P)
print("rouge_L: p: ")
print(_rouge_l_p)
sum = 0
for i in range(0,len(rouge_l_R)):
    sum = sum + rouge_l_R[i]
_rouge_l_r = sum/len(rouge_l_R)
print("rouge_L: Rr: ")
print(_rouge_l_r)'''